In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305219  [    0/60000]
loss: 2.293884  [ 6400/60000]
loss: 2.280672  [12800/60000]
loss: 2.273339  [19200/60000]
loss: 2.244366  [25600/60000]
loss: 2.218904  [32000/60000]
loss: 2.222589  [38400/60000]
loss: 2.194078  [44800/60000]
loss: 2.198004  [51200/60000]
loss: 2.161244  [57600/60000]
Test Error: 
 Accuracy: 47.5%, Avg loss: 2.159371 

Epoch 2
-------------------------------
loss: 2.168524  [    0/60000]
loss: 2.157704  [ 6400/60000]
loss: 2.111538  [12800/60000]
loss: 2.127182  [19200/60000]
loss: 2.066211  [25600/60000]
loss: 2.012923  [32000/60000]
loss: 2.027327  [38400/60000]
loss: 1.960733  [44800/60000]
loss: 1.965798  [51200/60000]
loss: 1.885501  [57600/60000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 1.896035 

Epoch 3
-------------------------------
loss: 1.923945  [    0/60000]
loss: 1.894772  [ 6400/60000]
loss: 1.798191  [12800/60000]
loss: 1.828300  [19200/60000]
loss: 1.715775  [25600/60000]
loss: 1.668861  [32000/600